# import statement

In [ ]:
# !pip install transformers peft --quiet #Removed the version specifications to solve compatibility issues
# !pip install torch --quiet
# !pip install peft --quiet
!pip install accelerate --quiet
!pip install datasets --quiet
!pip install sentencepiece --quiet
!pip install bitsandbytes --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 92.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM, LlamaForCausalLM, LlamaTokenizerFast
from peft import PeftModel  # 0.5.0

import os

import torch
import re
from transformers import LlamaTokenizerFast, LlamaForCausalLM
import pandas as pd

import json
import torch
import pandas as pd
import numpy as np

from tqdm import tqdm

import math


import warnings
warnings.filterwarnings("ignore")

from google.colab import drive # Link to your drive if you are not using Colab with GCP
drive.mount('/content/drive') # Models in this HW take a long time to get trained and make sure to save it here

Mounted at /content/drive


In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# Load Models

# model 1
base_model = "NousResearch/Llama-2-13b-hf"
peft_model = "FinGPT/fingpt-sentiment_llama2-13b_lora"
tokenizer = LlamaTokenizerFast.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
model = LlamaForCausalLM.from_pretrained(base_model, trust_remote_code=True, device_map = "cuda:0", load_in_4bit = True,)
model = PeftModel.from_pretrained(model, peft_model)
model = model.eval()

# model 2
# base_model = "NousResearch/Meta-Llama-3-8B"
# peft_model = "FinGPT/fingpt-mt_llama3-8b_lora"
# tokenizer = LlamaTokenizerFast.from_pretrained(base_model, trust_remote_code=True)
# tokenizer.pad_token = tokenizer.eos_token
# model = LlamaForCausalLM.from_pretrained(base_model, trust_remote_code=True, device_map = "cuda:0", load_in_4bit=True)
# model = PeftModel.from_pretrained(model, peft_model)
# model = model.eval()

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/196 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/462 [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/19.7M [00:00<?, ?B/s]

In [ ]:
from transformers import AutoConfig

config = AutoConfig.from_pretrained(base_model)

print(f"Max token limit: {config.max_position_embeddings}")

Max token limit: 4096


# Example

In [ ]:
prompt = [
'''Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}
Input: FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is aggressively pursuing its growth strategy by increasingly focusing on technologically more demanding HDI printed circuit boards PCBs .
Answer: ''',
'''Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}
Input: According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .
Answer: ''',
'''Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}
Input: A tinyurl link takes users to a scamming site promising that users can earn thousands of dollars by becoming a Google ( NASDAQ : GOOG ) Cash advertiser .
Answer: ''',
]

# Generate results
tokens = tokenizer(prompt, return_tensors='pt', padding=True, max_length=512).to("cuda:0") # Move tokens to the same device as the model
res = model.generate(**tokens, max_length=512)
res_sentences = [tokenizer.decode(i) for i in res]
out_text = [o.split("Answer: ")[1] for o in res_sentences]

# show results
for sentiment in out_text:
    print(sentiment)

# Output:
# positive
# neutral
# negative

 positive</s>
 neutral</s>
 negative</s>


In [ ]:
earnings_call_transcripts = [
"""Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}
Input: Our revenue has significantly declined this quarter due to macroeconomic headwinds and supply chain disruptions.
Answer: """,

"""Instruction: Evaluate the sentiment of this earnings call transcript. Please choose an answer from {negative/neutral/positive}.
Input: We experienced steady growth in sales and expect continued improvement in the next quarter.
Answer: """,

"""Instruction: Evaluate the sentiment of this earnings call transcript. Please choose an answer from {negative/neutral/positive}.
Input: Despite inflationary pressures, our profitability remained stable, and we are optimistic about future performance.
Answer: """,
]

# Generate results
tokens = tokenizer(earnings_call_transcripts, return_tensors='pt', padding=True, max_length=512).to("cuda:0") # Move tokens to the same device as the model
res = model.generate(**tokens, max_length=512)
res_sentences = [tokenizer.decode(i) for i in res]
out_text = [o.split("Answer: ")[1] for o in res_sentences]

# show results
for sentiment in out_text:
    print(sentiment)


 negative</s>
 neutral</s>
 neutral</s>


# generate results for earnings call (batch 0-1)

In [ ]:
def get_sentiments(text_list, tokenizer, model, batch_size=8, max_length=128, max_new_tokens=10):

    res_list = []
    num_batches = math.ceil(len(text_list) / batch_size)

    # for i in tqdm(range(num_batches), desc="Processing Sentiments in Batches"):
    for i in range(num_batches):
        torch.cuda.empty_cache()  # Free GPU memory before processing

        # Select batch
        batch_texts = text_list[i * batch_size : (i + 1) * batch_size]

        instruction = """You are an expert in stock analysis which is focus on earning call analysis.
        An earnings call is a teleconference, or webcast, in which a public company discusses the financial results of a reporting period.
        I need you to analysis following earning call text in order to do sentiment analysis.
        Respond with ONLY one word from {negative/neutral/positive}. Do NOT provide explanations, reasoning, or extra text. Just return one word."""

        # Create batch prompts
        batch_prompts = [
            f"Instruction: {instruction}.\nInput: {text}Answer: "
            for text in batch_texts
        ]

        # Tokenize batch
        tokens = tokenizer(batch_prompts, padding=True, truncation=True, max_length=max_length, return_tensors="pt").to("cuda:0")

        # Generate batch responses (but do NOT decode yet)
        with torch.no_grad():
            res = model.generate(**tokens, max_new_tokens=max_new_tokens)

        res_sentences = [tokenizer.decode(i, skip_special_tokens=True) for i in res]

        # Collect results for later processing
        res_list.extend(res_sentences)  # Move to CPU to prevent CUDA memory overflow

        # print(batch_prompts[0])
        # print(res_sentences[0])
        # try:
        #   sentiment = res_sentences[0].split("Answer:")[1].strip()
        #   print(sentiment)
        # except:
        #   print(res_sentences[0])

        # Explicitly delete tensors to free CUDA memory
        del tokens, res
        torch.cuda.empty_cache()

    return res_list  # Returns raw model outputs (to be post-processed later)

def process_raw_results(res_list, tokenizer):
    sentiments = []
    for list_sentences in tqdm(res_list):
        list_this_transcript = []
        for res_sentence in list_sentences:
            try:
                sentiment = res_sentence.split("Answer:")[1].strip()  # Extract only the sentiment
            except IndexError:
                sentiment = "ERROR"  # Handle cases where "Answer:" is missing
            list_this_transcript.append(sentiment)

        sentiments.append(list_this_transcript)

    return sentiments

In [ ]:
# DATA_DIR = "/content/drive/My Drive/11785 Coursework/DL Proj/data/"
# OUTPUT_DIR = "/content/drive/My Drive/11785 Coursework/DL Proj/result/"

# with open(DATA_DIR + "batch_transcriptid_dict.json", "r") as f:
#     batch_transcriptid_dict = json.load(f)

# list_transcript_name = [f'batch_{x}_transcriptcomponent.parquet' for x in range(2)]
# transcript_name = list_transcript_name[0]

In [ ]:
# df_temp = pd.read_parquet(DATA_DIR + transcript_name)
# df_temp = df_temp.sort_values(by=['transcriptid', 'transcriptcomponentid'])

In [ ]:
# for id, group in tqdm(df_temp.groupby('transcriptid'), desc="Processing Sentiments"):
#     result = get_sentiments(group['componenttext'].tolist(), tokenizer, model, batch_size=1, max_length=64, max_new_tokens=10)

In [ ]:
DATA_DIR = "/content/drive/My Drive/11785 Coursework/DL Proj/data/"
OUTPUT_DIR = "/content/drive/My Drive/11785 Coursework/DL Proj/result/"

with open(DATA_DIR + "batch_transcriptid_dict.json", "r") as f:
    batch_transcriptid_dict = json.load(f)

list_transcript_name = [f'batch_{x}_transcriptcomponent.parquet' for x in range(2)]

for transcript_name in list_transcript_name[0:1]:
    print(transcript_name)

    torch.cuda.empty_cache()

    df_temp = pd.read_parquet(DATA_DIR + transcript_name)
    df_temp = df_temp.sort_values(by=['transcriptid', 'transcriptcomponentid'])

    # for test
    # df_temp = df_temp.iloc[:200]

    df_grouped = df_temp.groupby('transcriptid')['componenttext'].apply(lambda x: ' '.join(x)).reset_index()

    list_result = []
    # for id, group in tqdm(df_temp.groupby('transcriptid'), desc="Processing Sentiments"):

    # iterate the rows
    for index, row in tqdm(df_grouped.iterrows(), total=df_grouped.shape[0], desc="Processing Sentiments"):
        # break
        # result = get_sentiments(row['componenttext'], tokenizer, model, batch_size=64, max_length=128, max_new_tokens=10)

        instruction = """You are an expert in stock analysis which is focus on earning call analysis.
        An earnings call is a teleconference, or webcast, in which a public company discusses the financial results of a reporting period.
        I need you to analysis following earning call text in order to do sentiment analysis.
        Respond with ONLY one word from {negative/neutral/positive}. Do NOT provide explanations, reasoning, or extra text. Just return one word."""

        # prompt = f"Instruction: {instruction}.\nInput: {row['componenttext'][0:2000]}\nAnswer: "
        prompt = f"Instruction: {instruction}.\nInput: {row['componenttext']}\nAnswer: "
        tokens = tokenizer(prompt, padding=True, truncation=True, max_length=4096, return_tensors="pt").to("cuda:0")

        with torch.no_grad():
            res = model.generate(**tokens, max_new_tokens=10)

        res_sentences = [tokenizer.decode(i, skip_special_tokens=True) for i in res]
        res_sentence = res_sentences[0]

        try:
            sentiment = res_sentence.split("Answer:")[1].strip()  # Extract only the sentiment
        except IndexError:
            sentiment = "ERROR"  # Handle cases where "Answer:" is missing

        list_result.append(sentiment)
        print(sentiment)
        # print(prompt)
        # print(res_sentences[0])
        # print("------------------------------------------------------")
    # break
    # sentiments = process_raw_results(list_result, tokenizer)
    # df_result = pd.DataFrame(columns=['transcriptid', 'sentiment'], data=list(zip(df_temp['transcriptid'].drop_duplicates().tolist(), sentiments)))
    # df_result.to_pickle(OUTPUT_DIR + "fingpt_sentiment_result_" + transcript_name.split(".")[0] + ".pkl")
    # print("save to ", OUTPUT_DIR + "fingpt_sentiment_result_" + transcript_name.split(".")[0] + ".pkl")
    # print("------------------------------------------------------")

batch_0_transcriptcomponent.parquet


Processing Sentiments:   0%|          | 1/5000 [00:03<4:12:53,  3.04s/it]

����������


Processing Sentiments:   0%|          | 2/5000 [00:06<4:20:04,  3.12s/it]

positive���������


...

Processing Sentiments:  96%|█████████▌| 4789/5000 [4:14:59<11:13,  3.19s/it]

����������


Processing Sentiments:  96%|█████████▌| 4790/5000 [4:15:02<11:14,  3.21s/it]

Pos��������trightarrow


Processing Sentiments:  96%|█████████▌| 4791/5000 [4:15:05<11:11,  3.21s/it]

In [ ]:
df_grouped['result'] = list_result
df_grouped = df_grouped.drop(columns=['componenttext'])
df_grouped.to_csv(OUTPUT_DIR + "fingpt_sentiment_result_" + transcript_name.split(".")[0] + ".csv")

Instruction: You are an expert in stock analysis which is focus on earning call analysis. 
        An earnings call is a teleconference, or webcast, in which a public company discusses the financial results of a reporting period. 
        I need you to analysis following earning call text in order to do sentiment analysis. 
        Respond with ONLY one word from {negative/neutral/positive}. Do NOT provide explanations, reasoning, or extra text. Just return one word..
Input: Good day, everyone, and welcome to the Adobe Fourth Quarter and Fiscal Year 2006 Earnings Call. Today’s call is being recorded. At this time, I would like to turn the call over to Mr. Mike Saviage, Vice President of Investor Relations. Please go ahead, sir.
 Good afternoon and thank you for joining us today. Joining me on the call are Bruce Chizen, our CEO and acting CFO; Shantanu Narayen, President and COO; and Rich Rowley, our new Vice President and Corporate Controller.
In the call today, we will discuss Adobe’s

In [ ]:
# for transcript_name in list_transcript_name:
#     print(transcript_name)
#     gc.collect()
#     torch.cuda.empty_cache()

#     df_temp = pd.read_parquet(DATA_DIR + transcript_name)
#     df_temp = df_temp.sort_values(by=['transcriptid', 'transcriptcomponentid'])

#     list_result = []
#     batch_counter = 0
#     sentiments_all = []
#     transcript_ids = []

#     for id, group in tqdm(df_temp.groupby('transcriptid'), desc="Processing Sentiments"):
#         result = get_sentiments(group['componenttext'].tolist(), tokenizer, model, batch_size=64, max_length=64, max_new_tokens=10)
#         list_result.append(result)
#         transcript_ids.append(id)
#         batch_counter += 1

#         if batch_counter % 500 == 0:
#             sentiments = process_raw_results(list_result, tokenizer)
#             df_partial_result = pd.DataFrame(columns=['transcriptid', 'sentiment'], data=list(zip(transcript_ids, sentiments)))
#             df_partial_result.to_pickle(OUTPUT_DIR + "fingpt_sentiment_result_" + transcript_name.split(".")[0] + f"_batch_{batch_counter}.pkl")
#             print("Saved batch at", OUTPUT_DIR + "fingpt_sentiment_result_" + transcript_name.split(".")[0] + f"_batch_{batch_counter}.pkl")

#             # Reset batch lists
#             list_result = []
#             transcript_ids = []

#     # Save remaining results if any
#     if list_result:
#         sentiments = process_raw_results(list_result, tokenizer)
#         df_final_result = pd.DataFrame(columns=['transcriptid', 'sentiment'], data=list(zip(transcript_ids, sentiments)))
#         df_final_result.to_pickle(OUTPUT_DIR + "fingpt_sentiment_result_" + transcript_name.split(".")[0] + "_final.pkl")
#         print("Saved final batch at", OUTPUT_DIR + "fingpt_sentiment_result_" + transcript_name.split(".")[0] + "_final.pkl")

#     print("------------------------------------------------------")


batch_0_transcriptcomponent.parquet


Processing Sentiments:   0%|          | 4/5000 [01:44<36:25:04, 26.24s/it]


KeyboardInterrupt: 

# test code

In [ ]:
# list_result = []
# for id, group in tqdm(df_temp.groupby('transcriptid'), desc="Processing Sentiments"):
#     result = get_sentiments(group['componenttext'].tolist(), tokenizer, model, batch_size=64, max_length=64, max_new_tokens=10)
#     list_result.append(result)

In [ ]:
# def get_sentiments(text_list, tokenizer, model, max_length=128, max_new_tokens=10):

#     sentiments = []
#     # for text in text_list:
#     for text in tqdm(text_list, desc="Processing: "):
#         torch.cuda.empty_cache()

#         # Create prompt
#         prompt = f"""Instruction: What is the sentiment of it? Please choose an answer from {{negative/neutral/positive}}\nInput: {text}Answer: """

#         # Tokenize input
#         tokens = tokenizer(prompt, padding=True, truncation=True, max_length=max_length, return_tensors="pt").to("cuda:0")

#         # Generate response
#         with torch.no_grad():
#             res = model.generate(**tokens, max_new_tokens=max_new_tokens)

#         # Decode response properly
#         res_sentence = tokenizer.decode(res[0], skip_special_tokens=True)

#         # ✅ Extract sentiment safely with exception handling
#         try:
#             sentiment = res_sentence.split("Answer:")[1].strip()
#         except IndexError:  # If "Answer:" is missing
#             sentiment = "ERROR"

#         sentiments.append(sentiment)

#         # ✅ Explicitly delete tensors to free CUDA memory
#         del tokens, res
#         torch.cuda.empty_cache()

#     return sentiments

# def get_sentiments(text_list, tokenizer, model, batch_size=8, max_length=128, max_new_tokens=10):
#     """
#     Processes a list of texts in batches to determine their sentiment.

#     Parameters:
#     - text_list (list): List of texts to analyze.
#     - tokenizer: Tokenizer instance.
#     - model: LLaMA model instance.
#     - batch_size (int): Number of samples per batch (default: 8).
#     - max_length (int): Maximum input length (default: 128).
#     - max_new_tokens (int): Maximum number of tokens the model should generate (default: 10).

#     Returns:
#     - sentiments (list): List of sentiment classifications {negative, neutral, positive} or "ERROR" if extraction fails.
#     """
#     sentiments = []
#     num_batches = math.ceil(len(text_list) / batch_size)

#     for i in tqdm(range(num_batches), desc="Processing Sentiments in Batches"):
#         torch.cuda.empty_cache()  # Free GPU memory before each batch

#         # Select batch
#         batch_texts = text_list[i * batch_size : (i + 1) * batch_size]

#         # Create batch prompts
#         batch_prompts = [
#             f"Instruction: What is the sentiment of it? Please choose an answer from {{negative/neutral/positive}}.\nInput: {text}Answer: "
#             for text in batch_texts
#         ]

#         # Tokenize batch
#         tokens = tokenizer(batch_prompts, padding=True, truncation=True, max_length=max_length, return_tensors="pt").to("cuda:0")

#         # Generate batch responses
#         with torch.no_grad():
#             res = model.generate(**tokens, max_new_tokens=max_new_tokens)

#         # Decode responses properly
#         res_sentences = [tokenizer.decode(i, skip_special_tokens=True) for i in res]

#         # ✅ Extract sentiments safely
#         for res_sentence in res_sentences:
#             try:
#                 sentiment = res_sentence.split("Answer:")[1].strip()
#             except IndexError:
#                 sentiment = "ERROR"
#             sentiments.append(sentiment)

#         # ✅ Explicitly delete tensors to free CUDA memory
#         del tokens, res
#         torch.cuda.empty_cache()

#     return sentiments


